In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from adam_prepare import titanic_pipeline
from sklearn.model_selection import train_test_split

### Acquire

In [18]:
titanic = prep_titanic(new_titanic_data())
titanic.head()

,survived,pclass,sex,sibsp,parch,fare,embark_town,alone
0,0,3,male,1,0,7.2500,Southampton,0
1,1,1,female,1,0,71.2833,Cherbourg,0
2,1,3,female,0,0,7.9250,Southampton,1
3,1,1,female,1,0,53.1000,Southampton,0
4,0,3,male,0,0,8.0500,Southampton,1


In [19]:
titanic['sex'] = titanic.sex.map({'male': 1, 'female': 0})
titanic['embark_town'] = titanic.embark_town.map({'Southampton': 0, 'Queenstown': 1, 'Cherbourg': 2})
titanic['survived'] = titanic.survived.astype(int)
titanic.head()

,survived,pclass,sex,sibsp,parch,fare,embark_town,alone
0,0,3,1,1,0,7.2500,0,0
1,1,1,0,1,0,71.2833,2,0
2,1,3,0,0,0,7.9250,0,1
3,1,1,0,1,0,53.1000,0,0
4,0,3,1,0,0,8.0500,0,1


### Prepare

In [20]:
#train, validate, split data
train, validate, test = split_data(titanic, 'survived')

### Isolate the target variable

In [22]:
# We know what our X and Y are, let's be expicit about defining them
X_train = train.drop(columns='survived')
y_train=train.survived

X_val = validate.drop(columns='survived')
y_val = validate.survived

X_test = test.drop(columns='survived')
y_test=test.survived

### Create the baseline 

In [23]:
# write a function to compute the baseline for a classification model

def establish_baseline(y_train):
    #  establish the value we will predict for all observations
    baseline_prediction = y_train.mode()

    # create a series of predictions with that value, 
    # the same length as our training set
    y_train_pred = pd.Series((baseline_prediction[0]), range(len(y_train)))

    # compute accuracy of baseline
    cm = confusion_matrix(y_train, y_train_pred)
    tn, fp, fn, tp = cm.ravel()

    accuracy = (tp+tn)/(tn+fp+fn+tp)
    return accuracy

In [24]:
establish_baseline(y_train)

0.6161290322580645

#### 1.Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)
